In [1]:
import pyomo.environ as pyomo
import matplotlib.pyplot as plt

In [2]:
prices = {
    0: 0.5,
    1: 0.6,
    2: 1.0,
    3: 1.0,
    4: 0.9,
    5: 1.1,
    6: 1.8,
    7: 1.5,
    8: 0.9,
    9: 0.8,
    10: 0.7,
    11: 1.0,
}

input_energy_schedule = {
    0: 0.0,
    1: 0.0,
    2: 0.0,
    3: 0.0,
    4: 0.3,
    5: 0.15,
    6: 0.15,
    7: 0.05,
    8: 0.05,
    9: 0.05,
    10: 0.0,
    11: 0.0,
}

In [3]:
model = pyomo.ConcreteModel()

In [4]:
# Parameters ######################
model.T = pyomo.Set(initialize=range(len(prices)))
# TODO: use input energy as a parameter
model.input_energy = pyomo.Param(model.T, initialize=input_energy_schedule)
model.price = pyomo.Param(model.T, initialize=prices)
# TODO: change SOC min and max to have a more dynamic system
model.soc_init = pyomo.Param(initialize=500.0)
model.soc_max = pyomo.Param(initialize=500.0)
model.soc_min = pyomo.Param(initialize=500.0)
model.sell_max = pyomo.Param(initialize=150.0)

In [5]:
# Variables #######################
model.v_sell = pyomo.Var(model.T, domain=pyomo.NonNegativeReals)
model.v_soc = pyomo.Var(model.T, domain=pyomo.NonNegativeReals)

In [7]:
# Constraints #####################
def sell_max_rule(model, t):
    return model.v_sell[t] <= model.sell_max


def soc_max_rule(model, t):
    return model.v_soc[t] <= model.soc_max


def soc_min_rule(model, t):
    return model.v_soc[t] >= model.soc_min


def energy_equlibrium_rule(model, t):
    if t == 0:
        return (
            model.soc_init + model.input_energy[t] * model.soc_max - model.v_sell[t]
            == model.v_soc[t]
        )
    else:
        return (
            model.v_soc[t - 1] + model.input_energy[t] * model.soc_max - model.v_sell[t]
            == model.v_soc[t]
        )


model.c_sell_max = pyomo.Constraint(model.T, rule=sell_max_rule)
model.c_soc_max = pyomo.Constraint(model.T, rule=soc_max_rule)
model.c_soc_min = pyomo.Constraint(model.T, rule=soc_min_rule)
model.c_energy_equlibrium = pyomo.Constraint(model.T, rule=energy_equlibrium_rule)

'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.constraint.IndexedConstraint'>). This
is usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.constraint.IndexedConstraint'>). This
is usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.constraint.IndexedConstraint'>). This
is usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
(type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown
with a new Component (type=<class
'pyomo.core.base.constraint.IndexedConstraint'>). This is usua